In [1]:
import sys
import numpy as np
import torch
import matplotlib.pyplot as plt
plt.style.use("dark_background")
%matplotlib inline

sys.path.append("../../src")
from models import MLP
from data import gen_rnd_ds
from losses import SquaredClassification
from block_analysis import block_hessian, curvature_effects, block_hessian_diag, block_hessian_off_diag

## Helpers

In [2]:
def get_model_ds_loss():    
    model =  MLP(inp_dim, hid_dim, out_dim, nlayer, bias, mode).cuda(device)
    ds = gen_rnd_ds(inp_dim, inp_mean, inp_var, 
                   out_dim, nsamp, device)
    loss_fn = SquaredClassification(out_dim)
    return model, ds, loss_fn

In [3]:
def pp(lr, delta, h, H):
    """
    """
    rel = abs((h.item() - H.sum().item()) / min(abs(H.sum().item()), abs(h.item())))
    print(f"LR {lr:.2E} \t ||Error={rel:.2E}  \t ||H={h.item():.2E} \t || BH={H.sum().item():.2E} \t || Delta={delta:.2E}")

def lr_range(model, ds, loss_fn, start=-8, stop=8, step=1, log_scale=True):
    """
    """
    for lr in range(start, stop, step):
        if log_scale:
            lr = 10**lr
        H = block_hessian(model, ds, loss_fn, lr)
        delta, h = curvature_effects(model, ds, loss_fn, lr)
        pp(lr, delta, h, H)

## Params

In [4]:
# Model
mode = "linear"
bias = False
nlayer = 3
inp_dim = 100
hid_dim = 100
out_dim = 10

# Data parameters
nsamp = 100
inp_mean = 0
inp_var = 1

# Others
device = 0

In [ ]:
model, ds, loss_fn = get_model_ds_loss()
lr_range(model, ds, loss_fn)